In [1]:
import helpers
import pandas as pd
import pyshark
import numpy as np
import sys, getopt, csv, os, datetime, time, logging

In [10]:
# Global Variables
ROUTER = '78:d2:94:4d:ab:3e'
WIFI_DEVICES = ['ec:1a:59:e4:fd:41', 'ec:1a:59:e4:fa:09',
                'ec:1a:59:e5:02:0d', '14:91:82:24:dd:34',
                '60:38:e0:ee:7c:e5', '14:91:82:cd:df:3d',
                'b4:75:0e:0d:94:65', 'b4:75:0e:0d:33:d5',
                '94:10:3e:2b:7a:55', '30:8c:fb:3a:1a:ad',
                'd0:73:d5:26:b8:4c', 'd0:73:d5:26:c9:27',
                'ac:84:c6:97:7c:cc', 'b0:4e:26:c5:2a:41',
                '70:4f:57:f9:e1:b8', ROUTER]

DEVICE_TYPE = {'ec:1a:59:e4:fd:41' : 'camera',
               'ec:1a:59:e4:fa:09' : 'camera',
               'ec:1a:59:e5:02:0d' : 'camera',
               '14:91:82:24:dd:34' : 'plug',
               '60:38:e0:ee:7c:e5' : 'plug',
               '14:91:82:cd:df:3d' : 'plug',
               'b4:75:0e:0d:94:65' : 'plug',
               'b4:75:0e:0d:33:d5' : 'plug',
               '94:10:3e:2b:7a:55' : 'plug',
               '30:8c:fb:3a:1a:ad' : 'camera',
               'd0:73:d5:26:b8:4c' : 'bulb', 
               'd0:73:d5:26:c9:27' : 'bulb',
               'ac:84:c6:97:7c:cc' : 'camera', 
               'b0:4e:26:c5:2a:41' : 'bulb',
               '70:4f:57:f9:e1:b8' : 'plug'}

SRC_DIR = './Source/'
DST_DIR = './Destination/'
TIMING_PKT_NUMBER = 25000
path_name = os.getcwd()
DATE = path_name[path_name.rindex('/')+1:]
PROC_TIME = "wifi_processing_time_" + DATE + ".csv"

In [3]:
def parse_packet(pkt, tgt_files_by_src, tgt_files_by_dst):
    try:
        pkt_dst = pkt.wlan.da
        pkt_src = pkt.wlan.sa
        
        if (pkt_src in WIFI_DEVICES) and (pkt_dst in WIFI_DEVICES):
            # Extract features
            pkt_time = pkt.frame_info.time_epoch
            pkt_len = pkt.length
            
            output = [pkt_time, pkt_len, pkt_src, pkt_dst]
            
            csv.writer(tgt_files_by_src[pkt_src]).writerow(output)
            csv.writer(tgt_files_by_dst[pkt_dst]).writerow(output)
    
    except AttributeError:
        print "ignored: ", pkt.number            

In [4]:
# Prepare writers
pt_file = open(PROC_TIME, 'w')
csv.writer(pt_file).writerow(["Unit", "Total Packets Processed", "Total Process Time", "Average Process Time"])
pt_file.close()

# Initialize counters
pkt_count = 0
total_time_processing = 0
total_time_start = time.time()

# Initialize dicts for each device
tgt_files_by_src = {}
tgt_files_by_dst = {}

In [5]:
# Initialize capture file 
cap = pyshark.FileCapture('/root/Documents/Thesis/PCAPS/wifi-01.cap', only_summaries=False)

# Get time of first packet
prev_pkt_time = cap[0].frame_info.time_epoch

# Initialize output folders
helpers.init_dirs()

Old ./Source deleted
Old ./Destination deleted


In [6]:
# Open output files for each Wi-Fi device
for device in WIFI_DEVICES:
    tgt_files_by_src[device] = open(SRC_DIR + device.replace(':', '.') + ".csv", 'a')
    tgt_files_by_dst[device] = open(DST_DIR + device.replace(':', '.') + ".csv", 'a')

In [7]:
# Go through each packet in capture, and store pertinent packets to csv files
for pkt in cap:
    pkt_count += 1
    
    time_start = time.time()
    if pkt.highest_layer == 'DATA':
        parse_packet(pkt, tgt_files_by_src, tgt_files_by_dst)
        total_time_processing += time.time() - time_start

total_time = time.time() - total_time_start

In [8]:
# Close files
for open_file in tgt_files_by_src.values():
    open_file.close()

for open_file in tgt_files_by_dst.values():
    open_file.close()

In [9]:
# Calculate time variables
final_time = time.time()
normalized_total_time = (TIMING_PKT_NUMBER * total_time) / pkt_count
normalized_processing_time = (TIMING_PKT_NUMBER * total_time_processing) / pkt_count

# Print time variables
print "Total number of packets processed: ", pkt_count
print "Total data processing time: ", total_time
print "Normalized total processing time per 25k packets: ", normalized_total_time
print "Total capture file processing time: ", total_time_processing
print "Normalized capture file processing time: ", normalized_processing_time

# Print out time metrics to csv
pt_file = open(PROC_TIME, 'a')
csv.writer(pt_file).writerow(["Packet capture iteration", pkt_count, total_time_processing, normalized_processing_time])
csv.writer(pt_file).writerow(["Component start and finish time", total_time_start, final_time, final_time-total_time_start])
pt_file.close()

Total number of packets processed:  146225
Total data processing time:  172.975336075
Normalized total processing time per 25k packets:  29.5734888143
Total capture file processing time:  1.6935532093
Normalized capture file processing time:  0.289545770098
